# Import libraries

In [ ]:
!pip install transformers==4.32.0 accelerate tiktoken einops transformers_stream_generator==0.0.4 scipy torchvision pillow tensorboard matplotlib

In [ ]:
import inspect
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch
torch.manual_seed(1)

# Instantiate model

In [ ]:
# Instantiate Qwen-VL-Chat model. via
# https://huggingface.co/Qwen/Qwen-VL-Chat
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", device_map="cuda", trust_remote_code=True).eval()

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions', exist_ok=True)

# Import datasets

## DISARM

In [ ]:
DISARM_test_all_captioned_Qwen_VL_Chat = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_Qwen-VL-Chat.csv')
DISARM_test_all_captioned_InternLM_XComposer = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_InternLM-XComposer.csv')
DISARM_test_all_captioned_llava = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_llava-v1.5-13b.csv')
DISARM_test_all_captioned_BLIP_2 = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_BLIP-2.csv')

# Set prompts and define a function to call the model

In [ ]:
def prompt_vanilla(entity):
    return inspect.cleandoc(f"""
    Does this meme harmfully target {entity}?
    yes: a social entity is subjected to some form of ill-treatment such as mental abuse, psycho-physiological injury, proprietary damage, emotional disturbance, or public image damage, based on their background (bias, social background, educational background, etc.) by a meme author.
    no: any benign mention (or depiction) of a social entity via humour, limerick, harmless pun or any content that does not cause distress.
    Constraint: Without using any other words, answer either yes or no.""")

In [ ]:
def prompt_with_OCR(entity, OCR):
    return inspect.cleandoc(f"""
    Text on this meme: \"\"\"
    {OCR}
    \"\"\"
    Does this meme harmfully target {entity}?
    yes: a social entity is subjected to some form of ill-treatment such as mental abuse, psycho-physiological injury, proprietary damage, emotional disturbance, or public image damage, based on their background (bias, social background, educational background, etc.) by a meme author.
    no: any benign mention (or depiction) of a social entity via humour, limerick, harmless pun or any content that does not cause distress.
    Constraint: Without using any other words, answer either yes or no.""")

In [ ]:
def prompt_with_caption(entity, caption):
    return inspect.cleandoc(f"""
    Description of this meme: \"\"\"
    {caption}
    \"\"\"
    Does this meme harmfully target {entity}?
    yes: a social entity is subjected to some form of ill-treatment such as mental abuse, psycho-physiological injury, proprietary damage, emotional disturbance, or public image damage, based on their background (bias, social background, educational background, etc.) by a meme author.
    no: any benign mention (or depiction) of a social entity via humour, limerick, harmless pun or any content that does not cause distress.
    Constraint: Without using any other words, answer either yes or no.""")

In [ ]:
# Use Qwen-VL-Chat for the inference. via
# https://huggingface.co/Qwen/Qwen-VL-Chat
def get_prediction(image, prompt):
    query = tokenizer.from_list_format([
        {'image': image},
        {'text': prompt},
    ])
    response, history = model.chat(tokenizer, query=query, history=None)
    return response

# Define function to clean responses

In [ ]:
def remap(x):
    x = x.lower()
    if x == 'yes':
        return 'harmful'
    elif x == 'no':
        return 'not harmful'
    elif 'yes' in x and 'no' not in x:
        return 'harmful'
    elif 'no' in x and 'yes' not in x and 'not' not in x:
        return 'not harmful'
    else:
        return None

# Call the `get_prediction` function and save inferences

GROUNDING [ABSENT] & PROMPT [VANILLA]

In [ ]:
DISARM_test_all_grounding_absent_prompt_vanilla = DISARM_test_all_captioned_Qwen_VL_Chat.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_vanilla_images = DISARM_test_all_grounding_absent_prompt_vanilla['image'].values
DISARM_test_all_grounding_absent_prompt_vanilla_entities = DISARM_test_all_grounding_absent_prompt_vanilla['target'].values
DISARM_test_all_grounding_absent_prompt_vanilla['prediction'] = [get_prediction(image, prompt_vanilla(entity)) for image, entity in zip(DISARM_test_all_grounding_absent_prompt_vanilla_images, DISARM_test_all_grounding_absent_prompt_vanilla_entities)]
DISARM_test_all_grounding_absent_prompt_vanilla.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_prompt[VANILLA]_prediction[Qwen-VL-Chat].csv', index=False)
DISARM_test_all_grounding_absent_prompt_vanilla['prediction'] = DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_vanilla['prediction'] = DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_vanilla['labels'].values, DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_vanilla['labels'].values, DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].values, labels=['harmful', 'not harmful']))

harmful        354
not harmful    258
Name: prediction, dtype: int64
0
0.44930222533541675
              precision    recall  f1-score   support

     harmful       0.47      0.53      0.50       316
 not harmful       0.43      0.37      0.40       296

    accuracy                           0.45       612
   macro avg       0.45      0.45      0.45       612
weighted avg       0.45      0.45      0.45       612



GROUNDING [ABSENT] & PROMPT [OCR]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_OCR = DISARM_test_all_captioned_Qwen_VL_Chat.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_OCR_images = DISARM_test_all_grounding_absent_prompt_with_OCR['image'].values
DISARM_test_all_grounding_absent_prompt_with_OCR_entities = DISARM_test_all_grounding_absent_prompt_with_OCR['target'].values
DISARM_test_all_grounding_absent_prompt_with_OCR_texts = DISARM_test_all_grounding_absent_prompt_with_OCR['extracted_text'].values
DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'] = [get_prediction(image, prompt_with_OCR(entity, text)) for image, entity, text in zip(DISARM_test_all_grounding_absent_prompt_with_OCR_images, DISARM_test_all_grounding_absent_prompt_with_OCR_entities, DISARM_test_all_grounding_absent_prompt_with_OCR_texts)]
DISARM_test_all_grounding_absent_prompt_with_OCR.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_prompt[OCR]_prediction[Qwen-VL-Chat].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'] = DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'] = DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_OCR['labels'].values, DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_OCR['labels'].values, DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].values, labels=['harmful', 'not harmful']))

harmful        328
not harmful    284
Name: prediction, dtype: int64
0
0.5150139216106233
              precision    recall  f1-score   support

     harmful       0.53      0.55      0.54       316
 not harmful       0.50      0.48      0.49       296

    accuracy                           0.52       612
   macro avg       0.52      0.52      0.52       612
weighted avg       0.52      0.52      0.52       612



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [Qwen_VL_Chat]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat = DISARM_test_all_captioned_Qwen_VL_Chat.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[Qwen-VL-Chat]_prompt[CAPTION]_prediction[Qwen-VL-Chat].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    521
harmful         91
Name: prediction, dtype: int64
0
0.44232961124025993
              precision    recall  f1-score   support

     harmful       0.57      0.16      0.26       316
 not harmful       0.49      0.87      0.63       296

    accuracy                           0.50       612
   macro avg       0.53      0.52      0.44       612
weighted avg       0.53      0.50      0.44       612



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [InternLM_XComposer]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer = DISARM_test_all_captioned_InternLM_XComposer.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_images = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_entities = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_captions = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_images, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_entities, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[InternLM-XComposer]_prompt[CAPTION]_prediction[Qwen-VL-Chat].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    574
harmful         38
Name: prediction, dtype: int64
0
0.38398597311513727
              precision    recall  f1-score   support

     harmful       0.58      0.07      0.12       316
 not harmful       0.49      0.95      0.64       296

    accuracy                           0.49       612
   macro avg       0.53      0.51      0.38       612
weighted avg       0.53      0.49      0.38       612



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [llava]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_llava = DISARM_test_all_captioned_llava.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_llava_images = DISARM_test_all_grounding_absent_prompt_with_caption_llava['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava_entities = DISARM_test_all_grounding_absent_prompt_with_caption_llava['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava_captions = DISARM_test_all_grounding_absent_prompt_with_caption_llava['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_llava_images, DISARM_test_all_grounding_absent_prompt_with_caption_llava_entities, DISARM_test_all_grounding_absent_prompt_with_caption_llava_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_llava.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[llava-v1.5-13b]_prompt[CAPTION]_prediction[Qwen-VL-Chat].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_llava['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_llava['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    567
harmful         45
Name: prediction, dtype: int64
0
0.4047820044103061
              precision    recall  f1-score   support

     harmful       0.64      0.09      0.16       316
 not harmful       0.49      0.95      0.65       296

    accuracy                           0.50       612
   macro avg       0.57      0.52      0.40       612
weighted avg       0.57      0.50      0.40       612



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [BLIP-2]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2 = DISARM_test_all_captioned_BLIP_2.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_images = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_entities = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_captions = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_images, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_entities, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[BLIP_2]_prompt[CAPTION]_prediction[Qwen-VL-Chat].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    416
harmful        196
Name: prediction, dtype: int64
0
0.516502808988764
              precision    recall  f1-score   support

     harmful       0.57      0.35      0.44       316
 not harmful       0.51      0.72      0.60       296

    accuracy                           0.53       612
   macro avg       0.54      0.54      0.52       612
weighted avg       0.54      0.53      0.51       612

